# Movie Recommendation System

By: Anat Jacobson, Stephanie Ciaccia and Charlie Jin

---

## Overview

Build a model that provides top 5 movie recommendations to a user, based on their ratings of other movies.

The MovieLens dataset is a "classic" recommendation system dataset, that is used in numerous academic papers and machine learning proofs-of-concept. You will need to create the specific details about how the user will provide their ratings of other movies, in addition to formulating a more specific business problem within the general context of "recommending movies".

# Business Problem

Add business problem here

# Data Understanding

In [1]:
#importing packages

In [195]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import datetime

import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
%matplotlib inline
import plotly.express as px
from matplotlib.ticker import StrMethodFormatter

from surprise.model_selection import cross_validate
from surprise import Dataset, Reader, accuracy
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline,  SVD, SVDpp, NMF, BaselineOnly, NormalPredictor
from surprise.model_selection import GridSearchCV, cross_validate, train_test_split
import numpy as np

from collections import Counter
from nltk.corpus import stopwords

import pandas as pd
from IPython.display import display


Function for printing long lists

In [3]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

### Data Source : Grouplens

The data used in our analysis was obtained from [Movie Lens]([http://movielens.org]), which consists of user ratings on movies using a 5-star scale. The dataset was collected between March 29, 1996, and September 24, 2018, and includes ratings from 610 users. In total, the dataset contains 100836 ratings and 3683 tags across 9742 movies.

Files from the dataset inclue `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`.

### Import CSVs, Merge, and Clean all Data Needed
#### 4 Datasets imported:

- Movies:
- Links:
- Ratings:
- Tags:

In [137]:
#importing datasets
ratings_df = pd.read_csv("data/ratings.csv")
movies_df = pd.read_csv("data/movies.csv")
links_df = pd.read_csv("data/links.csv")
tags_df = pd.read_csv("data/tags.csv")

In [138]:
#checking the shape of all dfs
print('movies_df: ', movies_df.shape)
print('links_df: ',links_df.shape)
print('ratings_df: ',ratings_df.shape)
print('tags_df: ',tags_df.shape)

movies_df:  (9742, 3)
links_df:  (9742, 3)
ratings_df:  (100836, 4)
tags_df:  (3683, 4)


In [140]:
#displaying the first 5 rows of each data frames
display(movies_df.head())
display(links_df.head())
display(ratings_df.head())
display(tags_df.head())

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [218]:
tag_group_df = tags_df.groupby(by="movieId").agg({"tag":" ".join})

In [219]:
#we would want to run a vectorizer and clean data
tag_group_df.value_counts()

tag                                                               
In Netflix queue                                                      109
Disney                                                                 15
aliens                                                                 10
Shakespeare                                                            10
Stephen King                                                            9
                                                                     ... 
death penalty Nun                                                       1
death penalty John Grisham                                              1
deafness                                                                1
darth vader luke skywalker space opera                                  1
"artsy" atmospheric gritty hallucinatory surreal visually stunning      1
Length: 1038, dtype: int64

### Inspecting ratings


In [141]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [142]:
ratings_df.userId.value_counts()

414    2698
599    2478
474    2108
448    1864
274    1346
       ... 
406      20
595      20
569      20
431      20
442      20
Name: userId, Length: 610, dtype: int64

### Inspecting movies

In [143]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [145]:
#splitting genre into multiple columns
genre_split = pd.DataFrame(movies_df.genres.str.split('|',6).tolist(),
                                 columns = ['genre_1','genre_2', 'genre_3',
                                            'genre_4', 'genre_5', 'genre_6', 'genre_7'])

#merging dataframes
movies_df = pd.merge(pd.DataFrame(movies_df), pd.DataFrame(genre_split), left_index=True, right_index=True)

#dropping genre columns
movies_df.drop(columns="genres", inplace=True)

In [222]:
genre_split

,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7
0,Adventure,Animation,Children,Comedy,Fantasy,None,None
1,Adventure,Children,Fantasy,None,None,None,None
2,Comedy,Romance,None,None,None,None,None
3,Comedy,Drama,Romance,None,None,None,None
4,Comedy,None,None,None,None,None,None
...,...,...,...,...,...,...,...
9737,Action,Animation,Comedy,Fantasy,None,None,None
9738,Animation,Comedy,Fantasy,None,None,None,None
9739,Drama,None,None,None,None,None,None
9740,Action,Animation,None,None,None,None,None


In [146]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genre_1  9742 non-null   object
 3   genre_2  6891 non-null   object
 4   genre_3  3673 non-null   object
 5   genre_4  1335 non-null   object
 6   genre_5  348 non-null    object
 7   genre_6  77 non-null     object
 8   genre_7  14 non-null     object
dtypes: int64(1), object(8)
memory usage: 685.1+ KB


In [147]:
#making year and title columns separate

#making year column
movies_df['year'] = movies_df['title'].str[-6:]

#making real_title column
movies_df['real_title'] = movies_df['title'].str[:-6]

#dropping original title column
movies_df = movies_df.drop(columns="title")

In [148]:
movies_df

,movieId,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,year,real_title
0,1,Adventure,Animation,Children,Comedy,Fantasy,None,None,(1995),Toy Story
1,2,Adventure,Children,Fantasy,None,None,None,None,(1995),Jumanji
2,3,Comedy,Romance,None,None,None,None,None,(1995),Grumpier Old Men
3,4,Comedy,Drama,Romance,None,None,None,None,(1995),Waiting to Exhale
4,5,Comedy,None,None,None,None,None,None,(1995),Father of the Bride Part II
...,...,...,...,...,...,...,...,...,...,...
9737,193581,Action,Animation,Comedy,Fantasy,None,None,None,(2017),Black Butler: Book of the Atlantic
9738,193583,Animation,Comedy,Fantasy,None,None,None,None,(2017),No Game No Life: Zero
9739,193585,Drama,None,None,None,None,None,None,(2017),Flint
9740,193587,Action,Animation,None,None,None,None,None,(2018),Bungo Stray Dogs: Dead Apple


### Inspecting links

The additional ids will be helpful so we can pull additional movie information from TMBD's api for the content based model.

In [149]:
links_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


### Inspecting tags

In [115]:
tags_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


### Merging files into one dataframe for cleaning

In [154]:
#merging ratings, movies, and links
final_movies = pd.merge(ratings_df, movies_df, on="movieId", how="left")
final_movies = pd.merge(final_movies, links_df, on="movieId", how="left")

In [155]:
final_movies

,userId,movieId,rating,timestamp,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,year,real_title,imdbId,tmdbId
0,1,1,4.0,964982703,Adventure,Animation,Children,Comedy,Fantasy,None,None,(1995),Toy Story,114709,862.0
1,1,3,4.0,964981247,Comedy,Romance,None,None,None,None,None,(1995),Grumpier Old Men,113228,15602.0
2,1,6,4.0,964982224,Action,Crime,Thriller,None,None,None,None,(1995),Heat,113277,949.0
3,1,47,5.0,964983815,Mystery,Thriller,None,None,None,None,None,(1995),Seven (a.k.a. Se7en),114369,807.0
4,1,50,5.0,964982931,Crime,Mystery,Thriller,None,None,None,None,(1995),"Usual Suspects, The",114814,629.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Drama,Horror,Thriller,None,None,None,None,(2017),Split,4972582,381288.0
100832,610,168248,5.0,1493850091,Action,Crime,Thriller,None,None,None,None,(2017),John Wick: Chapter Two,4425200,324552.0
100833,610,168250,5.0,1494273047,Horror,None,None,None,None,None,None,(2017),Get Out,5052448,419430.0
100834,610,168252,5.0,1493846352,Action,Sci-Fi,None,None,None,None,None,(2017),Logan,3315342,263115.0


# Data Preparation

To being, we will need to remove null values and split the genre column so this can be One Hot Encoded.

In [157]:
#saving as new dataframe for cleaning
movie_df = final_movies

In [359]:
#checking for null values
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100823 entries, 0 to 100835
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   user_id   100823 non-null  int64  
 1   movie_id  100823 non-null  int64  
 2   rating    100823 non-null  float64
 3   genre_1   100823 non-null  object 
 4   genre_2   100823 non-null  object 
 5   genre_3   100823 non-null  object 
 6   genre_4   100823 non-null  object 
 7   genre_5   100823 non-null  object 
 8   year      100823 non-null  object 
 9   title     100823 non-null  object 
 10  tmdbId    100823 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 11.7+ MB


In [160]:
#dropping genre columns that are mostly empty
movie_df = movie_df.drop(columns=["genre_6", "genre_7", "timestamp",
                                  "imdbId"])

In [161]:
#changing null value to 0 for genre id

genre_list = ["genre_2", "genre_3", "genre_4", "genre_5"]

for x in genre_list:
    movie_df[genre_list] = movie_df[genre_list].fillna(0)

In [162]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   userId      100836 non-null  int64  
 1   movieId     100836 non-null  int64  
 2   rating      100836 non-null  float64
 3   genre_1     100836 non-null  object 
 4   genre_2     100836 non-null  object 
 5   genre_3     100836 non-null  object 
 6   genre_4     100836 non-null  object 
 7   genre_5     100836 non-null  object 
 8   year        100836 non-null  object 
 9   real_title  100836 non-null  object 
 10  tmdbId      100823 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 9.2+ MB


In [357]:
#checking nulls
movie_df.isna().sum()

user_id     0
movie_id    0
rating      0
genre_1     0
genre_2     0
genre_3     0
genre_4     0
genre_5     0
year        0
title       0
tmdbId      0
dtype: int64

In [166]:
#checking datatypes
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   userId      100836 non-null  int64  
 1   movieId     100836 non-null  int64  
 2   rating      100836 non-null  float64
 3   genre_1     100836 non-null  object 
 4   genre_2     100836 non-null  object 
 5   genre_3     100836 non-null  object 
 6   genre_4     100836 non-null  object 
 7   genre_5     100836 non-null  object 
 8   year        100836 non-null  object 
 9   real_title  100836 non-null  object 
 10  tmdbId      100823 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 9.2+ MB


In [167]:
#dropping null values from dataframe
movie_df = movie_df.dropna()

In [168]:
#confirming null values and datatypes
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100823 entries, 0 to 100835
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   userId      100823 non-null  int64  
 1   movieId     100823 non-null  int64  
 2   rating      100823 non-null  float64
 3   genre_1     100823 non-null  object 
 4   genre_2     100823 non-null  object 
 5   genre_3     100823 non-null  object 
 6   genre_4     100823 non-null  object 
 7   genre_5     100823 non-null  object 
 8   year        100823 non-null  object 
 9   real_title  100823 non-null  object 
 10  tmdbId      100823 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 9.2+ MB


In [169]:
#renaming columns for modeling
movie_df = movie_df.rename(columns={"userId":"user_id"})
movie_df = movie_df.rename(columns={"movieId":"movie_id"})
movie_df = movie_df.rename(columns={"real_title":"title"})

#### Final Cleaned Dataset (movie_df)
Features are:
- Genre
- Rating
- Tag

In [170]:
#reviewing final cleaned dataset
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100823 entries, 0 to 100835
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   user_id   100823 non-null  int64  
 1   movie_id  100823 non-null  int64  
 2   rating    100823 non-null  float64
 3   genre_1   100823 non-null  object 
 4   genre_2   100823 non-null  object 
 5   genre_3   100823 non-null  object 
 6   genre_4   100823 non-null  object 
 7   genre_5   100823 non-null  object 
 8   year      100823 non-null  object 
 9   title     100823 non-null  object 
 10  tmdbId    100823 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 9.2+ MB


In [171]:
#exploring that dataset
movie_df.describe()

,user_id,movie_id,rating,tmdbId
count,100823.000000,100823.000000,100823.000000,100823.000000
mean,326.130823,19435.437737,3.501637,20105.462633
std,182.618176,35532.291269,1.042430,53274.143620
min,1.000000,1.000000,0.500000,2.000000
25%,177.000000,1199.000000,3.000000,712.000000
50%,325.000000,2991.000000,3.500000,6957.000000
75%,477.000000,8120.000000,4.000000,11635.000000
max,610.000000,193609.000000,5.000000,525662.000000


### Preliminary Analysis

After analyzing the distribution of the ratings, it is clear there is an inbalance. The majority of ratings are between **3-5**.

In [172]:
#making dataframe of the top 15 users
top_15_reviewers = pd.DataFrame(movie_df["user_id"].value_counts(ascending=False).head(15))

#resetting index
top_15_reviewers = top_15_reviewers.reset_index()

#renaming columns based on value_counts
top_15_reviewers = top_15_reviewers.rename(columns={"user_id":"review_count"})
top_15_reviewers = top_15_reviewers.rename(columns={"index":"user_id"})

In [596]:
# additional code for graphing with matplot lib incase ploty does not work

#making user_id a string for graphing
# top_15_reviewers['user_id'] = top_15_reviewers['user_id'].astype(str)
# #making sorted df for chart
# top_15_sorted = top_15_reviewers.sort_values('review_count')
# #making airbnb_rating a string
# grouped = top_15_sorted.groupby("user_id")["review_count"].sum()
# fig = plt.figure(figsize = (10, 8))
# # create a bar plot
# plt.bar(grouped.index, grouped.values, color="#f86424")
# # set the x-axis label
# plt.xlabel("User ID")
# # set the y-axis label
# plt.ylabel("Reviews")
# #adding title
# plt.title("Most Active Users")
# # show the plot
# plt.show()

In [174]:
#making dataframe of rating counts to compare distribution of ratings
top_ratings = pd.DataFrame(movie_df["rating"].value_counts(ascending=False).head(15))
top_ratings = top_ratings.reset_index()
top_ratings = top_ratings.rename(columns={"rating":"rating_count"})
top_ratings = top_ratings.rename(columns={"index":"rating"})

#making user_id a string for plotting
top_ratings['rating'] = top_ratings['rating'].astype(str)

In [175]:
top_ratings

,rating,rating_count
0,4.0,26816
1,3.0,20044
2,5.0,13209
3,3.5,13134
4,4.5,8551
5,2.0,7550
6,2.5,5550
7,1.0,2811
8,1.5,1791
9,0.5,1367


In [597]:
# # additional code for graphing with matplot lib incase ploty does not work

# #making bar graph of rating frequency
# grouped = top_ratings.groupby("rating")["rating_count"].sum()
# fig = plt.figure(figsize = (10, 8))
# # create a bar plot
# plt.bar(grouped.index, grouped.values, color="#f86424")
# # set the x-axis label
# plt.xlabel("Rating")
# # set the y-axis label
# plt.ylabel("Rating Count")
# #adding title
# plt.title("Rating Frequency")
# # show the plot
# plt.show()

In [591]:
#using plotly to plot the top reviewers
fig = px.bar(top_15_sorted, x="user_id", y="review_count")
fig.update_layout(title_text='Top Reviewers',
                  title_x=0.5,
                  xaxis_title="User Id",
                  yaxis_title="Review Count",
                 plot_bgcolor='white')
fig.update_traces(marker_color = "#f86424")
fig.show()

In [590]:
#using plotly to plot the top featurescolor=
fig = px.bar(top_ratings, x="rating", y="rating_count")
fig.update_layout(title_text='Rating Frequency',
                  title_x=0.5,
                  xaxis_title="Rating",
                  yaxis_title="Rating Count",
                 plot_bgcolor='white')
fig.update_traces(marker_color = "#f86424")
fig.show()

# Data Modeling - Recommendation System
To create a personalized recommendation system for our dataset, we will use a method called **user-based collaborative filtering**. This approach is based on the idea that people who have similar tastes or preferences in the past will have similar tastes in the future. This means that the model makes recommendations based on other similar user preferences.

To apply user-based collaborative filtering, the model will identify the movies that the user has rated in the past. It will then find other users who have rated those movies similarly to the target user. Once similar users have been found, the model will use their ratings to predict the target user's rating for movies they have not yet seen.

**Root Mean Square Error (RMSE)** score will be used as the main metric to determine the accuracy of the predictive model. The RMSE measures the difference between the actual and prediting ratings. The movie ratings are on a scale of 1-5.

### Surprise Data
To make this model, we will be using the python package Surprise. This is a scikit tool that uses a range of algorithms made up of matrix factorization-based methods for collaborative filtering. 

To begin, we will make new dataframe from our final cleaned dataframe with three columns that include user **id, ratings, and movie ids.**

In [181]:
#dropping columns that aren't needed for Surprise (since )
surprise_df = movie_df.drop(columns=["title", "genre_1", "genre_2", "genre_3", "genre_4",
                                    "genre_5", "tmdbId", "year"])

In [182]:
#checking final dataframe
surprise_df

,user_id,movie_id,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [259]:
surprise_df['user_id'].value_counts()

414    2698
599    2478
474    2108
448    1863
274    1346
       ... 
406      20
595      20
569      20
431      20
442      20
Name: user_id, Length: 610, dtype: int64

### Surprise Data and Train Test Split

In [265]:
from surprise import Reader, Dataset

reader = Reader(rating_scale=(1, 5))

#loading final dataset
data = Dataset.load_from_df(surprise_df, reader)

#spltting into train and test
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [191]:
#looking at number of users
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items)

Number of users:  610 

Number of items:  8951


## Baseline Model

In [196]:
# Instantiate the model
baseline = NormalPredictor()

#fitting model
baseline.fit(trainset)

# making prediction on testset
predictions = baseline.test(testset)

# Save RMSE score
baseline_normal = accuracy.rmse(predictions)

RMSE: 1.4310


In [537]:
#saving normal rmse
test_baseline_normal_rmse = 1.4310

The Normal Predictor baseline model's Root Mean Squared Error (RMSE) tells us that our predicted rating is **1.43** points away from the actual rating.

In [197]:
# Instantiate and fit model
baseline2 = BaselineOnly()

#fitting model
baseline2.fit(trainset)

# making prediction on testset
predictions = baseline2.test(testset)

# Save RMSE score
baseline_only = accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.8725


In [538]:
#saving baseline rmse
test_baseline_only_rmse = 0.8725

The Baseline Only model's Root Mean Squared Error (RMSE) tells us that our predicted rating is **.875** points away from the actual rating. This is an improvement from the first model.

## Model #1 - SVD

We will be using Singular Value Decomposition (SVD) to reduce the dimensionality of our matrix.

In [202]:
# Cross validate a basic SVD with no hyperparameter tuning expecting sub-par results
svd_basic = SVD(random_state=42)

results = cross_validate(svd_basic, data, measures=['RMSE'], cv=3, n_jobs = -1, verbose=True)

Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8783  0.8784  0.8811  0.8793  0.0013  
Fit time          3.14    3.11    3.16    3.14    0.02    
Test time         0.16    0.16    0.16    0.16    0.00    


In [203]:
# Fit to trainset and predict on the testset for evaluation
svd_basic.fit(trainset)

predictions = svd_basic.test(testset)

svd_simple = accuracy.rmse(predictions)

RMSE: 0.8734


In [536]:
#saving simmple rmse
test_svd_simple_rmse = 0.8734

The SVD performed worse than the base model. Further tuning of the hyperparameters will need to be made.

## SVD Grid Search

With a SVD Grid Search, we will adjust the following hyperparameters:

- **n_factors:** Increasing the number of n_factors (latent features) will make the model more complex and could help with performance on the testing data. A number that is too high, could result in overfitting. 

- **n_epochs:** This adjusts the number of iterations that are run on the data. A value that is too low will underfit the model, while a value that is too high could result in overfitting. Since this is a large dataset, we will most likely end up with a higher number.

### Grid Search #1

In [208]:
#test grid search
params = {'n_factors': [10, 50, 100, 150, 200],
          'n_epochs': [10, 20],
          'init_mean': [-0.5, 0.5], 
          'reg_all': [.05, .01, 0.02],
         'biased': [True, False]}

g_s_svd = GridSearchCV(SVD, param_grid=params, cv=5, refit=True)

g_s_svd.fit(data)
g_s_svd.best_params['rmse']

{'n_factors': 10,
 'n_epochs': 20,
 'init_mean': 0.5,
 'reg_all': 0.02,
 'biased': False}

In [210]:
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.8808879468723733, 'mae': 0.6758993339989241}
{'rmse': {'n_factors': 10, 'n_epochs': 20, 'init_mean': 0.5, 'reg_all': 0.02, 'biased': False}, 'mae': {'n_factors': 10, 'n_epochs': 20, 'init_mean': -0.5, 'reg_all': 0.01, 'biased': False}}


In [271]:
# instantiating SVD with best hyperparameters from gridsearch
g_s_svd = SVD(n_factors=10,n_epochs=20, init_mean=0.5, reg_all=0.02, biased=False)

# fit on trainset and make predictions using testset
g_s_svd.fit(trainset)
predictions = g_s_svd.test(testset)
g_s_svd_1 = accuracy.rmse(predictions)

RMSE: 0.8771


In [535]:
#saving test rmse
test_g_s_svd_1_params = {'n_factors': 10,'n_epochs': 20,'init_mean': 0.5,'reg_all': 0.02, 'biased': False}
test_g_s_svd_1_rmse = 0.8771

## Grid Search #2

In [215]:
#test grid search
params_2 = {'n_factors': [10, 50, 100, 200, 300],
          'reg_all': [.05, .01, 0.02, .005],
         'n_epochs': [10, 20, 30]}

g_s_svd_2 = GridSearchCV(SVD, param_grid=params_2, cv=3, refit=True)

g_s_svd_2.fit(data)
g_s_svd_2.best_params['rmse']

{'n_factors': 10,
 'n_epochs': 10,
 'init_mean': 0.5,
 'reg_all': 0.01,
 'biased': False}

In [223]:
print(g_s_svd_2.best_score)
print(g_s_svd_2.best_params)

{'rmse': 0.8840175621942711, 'mae': 0.6787384404177316}
{'rmse': {'n_factors': 10, 'n_epochs': 10, 'init_mean': 0.5, 'reg_all': 0.01, 'biased': False}, 'mae': {'n_factors': 10, 'n_epochs': 10, 'init_mean': 0.5, 'reg_all': 0.01, 'biased': False}}


In [272]:
#saving best params
grid_2_best_params = g_s_svd_2.best_params['rmse']

In [273]:
grid_2_best_params

{'n_factors': 10,
 'n_epochs': 10,
 'init_mean': 0.5,
 'reg_all': 0.01,
 'biased': False}

In [275]:
# instantiating SVD with best hyperparameters from gridsearch
g_s_svd_2 = SVD(n_factors=10, n_epochs=10, init_mean=0.5, reg_all=0.01, biased=False)

# fit on trainset and make predictions using testset
g_s_svd_2.fit(trainset)
predictions = g_s_svd_2.test(testset)
g_s_svd_2_rmse = accuracy.rmse(predictions)

RMSE: 0.8791


In [544]:
#saving test rmse
test_g_s_svd_2_params = {'n_factors': 10, 'n_epochs': 10, 'init_mean': 0.5, 'reg_all': 0.01, 'biased': False}
test_g_s_svd_2_rmse = 0.8791

### Grid Search #3

In [525]:
#test grid search
params_3 = {'n_factors': [50, 100, 200],
          'reg_all': [0.02, .05],
         'n_epochs': [10, 20, 30]}


g_s_svd_3 = GridSearchCV(SVD, param_grid=params_3, cv=5, refit=True)

g_s_svd_3.fit(data)
print(g_s_svd_3.best_params['rmse'])

{'n_factors': 100, 'reg_all': 0.05, 'n_epochs': 30}


In [528]:
print(g_s_svd_3.best_score['rmse'])

0.8623815254788681


In [526]:
#saving best params
g_s_svd_3_best_params = {'n_factors': 100, 'reg_all': 0.05, 'n_epochs': 30}

In [529]:
# instantiating SVD with best hyperparameters from gridsearch
g_s_svd_3 = SVD(n_factors=100, n_epochs=30, reg_all=0.05)

# fit on trainset and make predictions using testset
g_s_svd_3.fit(trainset)
predictions = g_s_svd_3.test(testset)
g_s_svd_3_rmse = accuracy.rmse(predictions)

RMSE: 0.8630


In [545]:
#saving test rmse and params
test_g_s_svd_3_params = {'n_factors': 100, 'reg_all': 0.05, 'n_epochs': 30}
test_g_s_svd_3_rmse = 0.8630

### Grid Search #4

In [515]:
#test grid search
params_4 = {'n_factors': [400, 500, 600],
          'reg_all': [0.02, .05],
           'n_epochs': [70, 80, 90]}

g_s_svd_4 = GridSearchCV(SVD, param_grid=params_6, cv=3, refit=True)

g_s_svd_4.fit(data)
g_s_svd_4.best_params['rmse']

{'n_factors': 400, 'reg_all': 0.05, 'n_epochs': 90}

In [516]:
print(g_s_svd_4.best_score['rmse'])

0.8621651917446677


In [517]:
#saving params and rmse score
best_model_rmse = 0.8621651917446677
best_model_params = {'n_factors': 400, 'reg_all': 0.05, 'n_epochs': 90}

In [518]:
# instantiating SVD from grid search params
svd_best_model = SVD(n_factors=400, reg_all=0.05, n_epochs=90)

# fit on trainset and make predictions using testset
svd_best_model.fit(trainset)
predictions = svd_best_model.test(testset)
svd_gs_best = accuracy.rmse(predictions)

RMSE: 0.8545


In [546]:
#saving test rmse
test_svd_gs_best_params = {'n_factors': 400, 'reg_all': 0.05, 'n_epochs': 90}
test_svd_gs_best_rmse = 0.8545

### NFM Grid Search

The NFM Grid Search performed worse than all other models in terms of RMSE.

In [423]:
# New hyperparameter dictionary for nmf model
nmf_param_grid = {'biased':[True, False],
                  'n_factors':[100, 500],
                  'n_epochs': [20, 25, 50]}
nmf_gs_model = GridSearchCV(NMF, param_grid=nmf_param_grid, cv=3, joblib_verbose=10, return_train_measures=True)

# Fit and return the best hyperparameters
nmf_gs_model.fit(data)
nmf_gs_model.best_params['rmse']

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   11.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   17.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   24.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   31.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   37.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:   49.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 15.9min finished


{'biased': False, 'n_factors': 100, 'n_epochs': 50}

In [530]:
nmf_gs_model.best_score['rmse']

1.0316376118114208

In [531]:
#saving params and best rmse
nmf_gs_model_rmse = 1.0316376118114208
nmf_gs_model_best_params = {'biased': False, 'n_factors': 100, 'n_epochs': 50}

In [425]:
# instantiating NFM
nfm_model = NMF(biased=False, n_factors=100, n_epochs=50)

# Fit on trainset and make predictions using testset to return RMSE metric
nfm_model.fit(trainset)
predictions = nfm_model.test(testset)
nfm_model_1 = accuracy.rmse(predictions)

RMSE: 1.0341


In [547]:
#saving test nfm model rmse
test_nfm_model_1_params = {'biased': False, 'n_factors': 100, 'n_epochs': 50}
test_nfm_model_1_rmse = 1.0341

# Results

#### Best Model

We used Singular Value Decomposition (SVD) to create our final model. This is a collaborative filtering method that identifies relationship between users movie ratings to make personzlied recommendations for users, based on their previous ratings.

Our best model was the final SVD Grid Search that gave us a **RMSE of .85**. 

- n_factors = 400
- reg_all = .05
- n_epochs = 90

In [540]:
#making dataframe of all RMSE scores for all models

model_name = ["Baseline Only", "SVD Basic", "SVD GS1", "SVD GS2", "SVD GS3", "SVD Best Model"]
rmse_score_model = [test_baseline_only_rmse, test_svd_simple_rmse,
                    test_g_s_svd_1_rmse, test_g_s_svd_2_rmse, test_g_s_svd_3_rmse, test_svd_gs_best_rmse]

model_df = pd.DataFrame()
model_df['RMSE'] = rmse_score_model
model_df['Model'] = model_name
model_df = model_df.sort_values(by="RMSE")

In [541]:
model_df

,RMSE,Model
5,0.8545,SVD Best Model
4,0.8630,SVD GS3
0,0.8725,Baseline Only
1,0.8734,SVD Basic
2,0.8771,SVD GS1
3,0.8791,SVD GS2


In [594]:
import plotly.express as px

#using plotly to plot the model RMSE scores
fig = px.bar(model_df, x="Model", y="RMSE", text=y, text_auto='.2')

# marker color for all bars
colors = ['#f86424'] * len(x)

#changing color of best model
colors[0] = '#24b8f8'

fig.update_layout(
    plot_bgcolor='white',
    title_text='Surprise Model Comparison',
    title_x=0.5,
    xaxis_title="Model",
    yaxis_title="RMSE",
    yaxis=dict(
        tick0=0.8,
        dtick=0.01,
        tickformat='.2f',
        range=[0.83, max(y)*1.01]
    )
)

fig.update_traces(marker_color=colors, textposition="outside")

fig.show()

## Function

Below we will be building a function called "ind_recommend_movies()" which will take in a user ID and return a specific number of reccommended movies based on the predicted rating the user would give. This function is built of of our best model from above XXXX. The code for this function has been adapted from Daniel Burdenos Capstone Project . To beging this function I will be creating a new "user" dataframe for ease of use and adding a new column to the movie_df called "average_rating"

user_df: this dataframe only contains the user_id and movie_id with the user_id being the index

In [429]:
#saving new dataframe with only user information
user_df = surprise_df.reset_index()
user_df.set_index('user_id', inplace = True)
user_df.drop(columns = 'rating', inplace =True)
user_df.head()

,index,movie_id
user_id,,
1,0,1
1,1,3
1,2,6
1,3,47
1,4,50
...,...,...
610,100831,166534
610,100832,168248
610,100833,168250


Creating a new column called "average_rating" that takes all the ratings for a given movie and shows the average for it. This will be used for the user to understand how their expected rating compares to the average rating for a given movie.

In [489]:
#creating new column
movie_df['Average_rating'] = movie_df.groupby('movie_id')['rating'].mean()
#checking the creation of column
movie_df

,user_id,movie_id,rating,genre_1,genre_2,genre_3,genre_4,genre_5,year,title,tmdbId,Average_rating
0,1,1,4.0,Adventure,Animation,Children,Comedy,Fantasy,(1995),Toy Story,862.0,NaN
1,1,3,4.0,Comedy,Romance,0,0,0,(1995),Grumpier Old Men,15602.0,3.920930
2,1,6,4.0,Action,Crime,Thriller,0,0,(1995),Heat,949.0,3.431818
3,1,47,5.0,Mystery,Thriller,0,0,0,(1995),Seven (a.k.a. Se7en),807.0,3.259615
4,1,50,5.0,Crime,Mystery,Thriller,0,0,(1995),"Usual Suspects, The",629.0,2.357143
...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,Drama,Horror,Thriller,0,0,(2017),Split,381288.0,NaN
100832,610,168248,5.0,Action,Crime,Thriller,0,0,(2017),John Wick: Chapter Two,324552.0,NaN
100833,610,168250,5.0,Horror,0,0,0,0,(2017),Get Out,419430.0,NaN
100834,610,168252,5.0,Action,Sci-Fi,0,0,0,(2017),Logan,263115.0,NaN


Below we are defining the function:

In [495]:
def ind_recommend_movies():
    
    user = int(input('user_id: '))
    n_recs = int(input('How many movie recommendations do you want? '))
    
    have_rated = list(user_df.loc[user, 'movie_id'])
    not_rated = movie_df.copy()
    not_rated = not_rated.drop_duplicates(subset=['movie_id'])
    not_rated.reset_index(inplace = True)
    not_rated['est_rating'] = not_rated['movie_id'].apply(lambda x: svd_best_model.predict(user, x).est)
    not_rated.sort_values(by = 'est_rating', ascending = False, inplace = True)
    not_rated = not_rated.rename(columns={'movie_id':'Movie',
                               'genre_1':'Genre','genre_2':'Genre', 'genre_3':'Genre',
                                           'genre_4':'Genre', 'genre_5':'Genre', 
                                         'title':'Title', 'est_rating':'Predicted Rating'})
    not_rated = not_rated.drop(columns=["tmdbId", 'rating', 'index', 'user_id'])
    return not_rated.head(n_recs)

In [496]:
ind_recommend_movies()

user_id: 450
How many movie recommendations do you want? 5


,Movie,Genre,Genre,Genre,Genre,Genre,year,Title,Average_rating,Predicted Rating
365,1719,Drama,0,0,0,0,(1997),"Sweet Hereafter, The",2.416667,4.914297
4886,2924,Action,Comedy,0,0,0,(1978),Drunken Master (Jui kuen),NaN,4.874375
7,110,Action,Drama,War,0,0,(1995),Braveheart,3.185185,4.824978
2038,2160,Drama,Horror,Thriller,0,0,(1968),Rosemary's Baby,4.187500,4.823939
1796,3972,Action,Comedy,0,0,0,(1994),"Legend of Drunken Master, The (Jui kuen II)",NaN,4.819149


In [513]:
movie_df.loc[movie_df['Average_rating'].isna()]

,user_id,movie_id,rating,genre_1,genre_2,genre_3,genre_4,genre_5,year,title,tmdbId,Average_rating
0,1,1,4.0,Adventure,Animation,Children,Comedy,Fantasy,(1995),Toy Story,862.0,NaN
33,1,592,4.0,Action,Crime,Thriller,0,0,(1989),Batman,268.0,NaN
35,1,596,5.0,Animation,Children,Fantasy,Musical,0,(1940),Pinocchio,10895.0,NaN
37,1,648,3.0,Action,Adventure,Mystery,Thriller,0,(1996),Mission: Impossible,954.0,NaN
51,1,1024,5.0,Animation,Children,Musical,0,0,(1945),"Three Caballeros, The",15947.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,Drama,Horror,Thriller,0,0,(2017),Split,381288.0,NaN
100832,610,168248,5.0,Action,Crime,Thriller,0,0,(2017),John Wick: Chapter Two,324552.0,NaN
100833,610,168250,5.0,Horror,0,0,0,0,(2017),Get Out,419430.0,NaN
100834,610,168252,5.0,Action,Sci-Fi,0,0,0,(2017),Logan,263115.0,NaN


# Conclusions

Pick a couple users and look at the ratings and see if the recommendation system returns are in line with it.